# Создание и обучение модели LSTM на векторном представлении аккордов

## Подключение библиотек

In [ ]:
import collections
import datetime
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf
import json
from IPython import display
from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple
import os

## Задаем стандарный random seed и проверяем GPU

In [ ]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Sampling rate for audio playback
_SAMPLING_RATE = 16000


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


## Загружаем csv данные

In [ ]:
train = pd.read_csv('Chord2vec train filt.csv').drop('Unnamed: 0', axis = 1)
test = pd.read_csv('Chord2vec test filt.csv').drop('Unnamed: 0', axis = 1)
k = open('Decode.json')
Decode = json.load(k)
Code = {v:k for k, v in Decode.items()}

## Конвертируем csv в numpy

### Создаем np.array размерностью (None, 101, 30), где 101 - размер вектора + длительность данного аккорда

In [ ]:
def split_array(array, length):
    return list([list(array[i:i+length]) for i in range(0, len(array), length)])

In [ ]:
trainData = []
trainY = []
for i in train.to_numpy():
    slices = split_array(i[:-2], 101)
    for slice_ in slices:
        trainData.append([slice_])
    trainY.append([i[-2], i[-1]])

In [ ]:
testData = []
testY = []
for i in test.to_numpy():
    slices = split_array(i[:-2], 101)
    for slice_ in slices:
        testData.append([slice_])
    testY.append([i[-2], i[-1]])

### Cохраняем полученные target`ы и данные в np.array

In [ ]:
trainData = np.array(trainData)
trainY = np.array(trainY)
testData = np.array(testData)
testY = np.array(testY)

### Меняем размерность np.array

In [ ]:
trainData = np.array([item[0] for item in trainData])
testData = np.array([item[0] for item in testData])

### кодируем target`ы загруженной ранее кодировкой

In [ ]:
for i in range(len(trainY),29):
    trainY[i][0] = float(Code[trainY[i][0]])
for i in range(len(testY)):
    testY[i][0] = float(Code[testY[i][0]])

### представляем в виде one hot encoding

In [ ]:
key_order =  [f"chord_{i}" for i in range(100)] + ['duration']
np.array(key_order).shape

### Создаем tf.Dataset

In [ ]:
notes_ds = tf.data.Dataset.from_tensor_slices(trainData)
notes_ds.element_spec

In [ ]:
def create_sequences(
    dataset: tf.data.Dataset, 
    seq_length: int,
    vocab_size = 100,
) -> tf.data.Dataset:
    seq_length = seq_length

    windows = dataset.window(seq_length, shift=1, stride=1,
                              drop_remainder=True)
    
    flatten = lambda x: x.batch(seq_length, drop_remainder=True)
    sequences = windows.flat_map(flatten)

    def scale_pitch(x):
        x = x/([1] * vocab_size + [10.0])
        return x

    def split_labels(sequences):
        inputs = sequences[:-1]
        labels_dense = sequences[-1]
        labels = {key:labels_dense[i] for i,key in enumerate(key_order)}
  
        return scale_pitch(inputs), labels
  
    return sequences.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
seq_length = 29
seq_ds = create_sequences(notes_ds, seq_length)
seq_ds.element_spec

### Проверка данных после преобразования

In [ ]:
for seq, target in seq_ds.take(2):
    print('sequence shape:', seq.shape)
    print('sequence elements (first 10):', seq)
    print()
    print('target:', target)

### Кешируем данные для более быстрого доступа, также перемешиваем и разделяем на batch

In [ ]:
batch_size = 128
buffer_size = len(train) - batch_size * 49
train_ds = (seq_ds
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE))

### Написали свою функцию потерь, чтобы контролировать отрицательные значения

In [ ]:
def mse_with_positive_pressure(y_true: tf.Tensor, y_pred: tf.Tensor):
    mse = (y_true - y_pred) ** 2
    positive_pressure = 10 * tf.maximum(-y_pred, 0.0)
    return tf.reduce_mean(mse + positive_pressure)

## Инициализация модели и обучение

### Создаем модель загружаем функции ошибки, веса по параметрам и выводим данные о ней

In [ ]:
input_shape = (seq_length, 2)
learning_rate = 0.05

inputs = tf.keras.Input(input_shape)
x = tf.keras.layers.LSTM(256)(inputs)

# x = tf.keras.layers.Dense(512, activation='relu')(x)

outputs = {
  'chord': tf.keras.layers.Dense(193, name='chord')(x),
  'duration': tf.keras.layers.Dense(5, name='duration')(x),
}

model = tf.keras.Model(inputs, outputs)

loss = {
      'chord': tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True),
      'duration': mse_with_positive_pressure,
}

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(
    loss=loss,
    loss_weights={
        'pitch': 1,
        'duration': 1.5,
    },
    optimizer=optimizer,
)

model.summary()

### Обучение модели

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./training_checkpoints/ckpt_{epoch}',
        save_weights_only=True),
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=25,
        verbose=1,
        restore_best_weights=True),
]

In [ ]:
epochs = 5000

history = model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
)

### Построение графика ошибки по эпохам

In [ ]:
plt.plot(history.epoch, history.history['loss'], label='total loss')
plt.show()

### Cозранение модели для будующего использования

In [ ]:
model.save("path/to/my_model.h5")

##  Попытка обучить модель с разными ветками на параметры

In [ ]:
input_shape = (7, 101)
learning_rate = 0.001

inputs = tf.keras.Input(input_shape)
# x = tf.keras.layers.LSTM(100)(inputs)

# x = tf.keras.layers.Dense(128, activation='relu')(x)
# x = tf.keras.layers.Dense(256, activation='relu')(x)
# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dense(256, activation='relu')(x)

# Создание первой ветви модели для целевого значения "chord"
chord_branch = tf.keras.layers.LSTM(64)(inputs)
chord_branch = tf.keras.layers.Dense(64, activation='relu')(chord_branch)
chord_output = tf.keras.layers.Dense(193, activation='softmax', name='chord')(chord_branch)

# Создание второй ветви модели для целевого значения "duration"
duration_branch = tf.keras.layers.LSTM(64)(inputs)
duration_branch = tf.keras.layers.Dense(64, activation='relu')(duration_branch)
duration_output = tf.keras.layers.Dense(1, name='duration')(duration_branch)


outputs = {
  'chord': tf.keras.layers.Dense(1,activation='softmax', name='chord')(x),
  'duration': tf.keras.layers.Dense(1, name='duration')(x),
}

modelChord2 = tf.keras.Model(inputs=inputs, outputs=[chord_output, duration_output])

loss = {
      'chord': tf.keras.losses.CategoricalCrossentropy(),
      'duration': mse_with_positive_pressure,
}

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

modelChord2.compile(loss=loss,metrics={'chord': 'accuracy', 'duration':'mae'},loss_weights={
        'chord': 1,
        'duration':0.5,
    },
    optimizer=optimizer)

modelChord2.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./training_checkpoints/ckpt_{epoch}',
        save_weights_only=True),
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=50,
        verbose=1,
        restore_best_weights=True),
]

In [ ]:
epochs = 5000

history = modelChord2.fit(
    x = chord_ds,
    y = {
    'chord': chord_target,
    'duration': dur_target
    },
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.35
)

In [ ]:
plt.plot(history.epoch, history.history['loss'], label='total loss')
plt.show()

## График по всем Loss`ам

In [ ]:
loss = history.history['loss']  # Значения ошибки для первого целевого параметра
loss_target2 = history.history['chord_loss']  # Значения ошибки для второго целевого параметра
loss_target3 = history.history['duration_loss']
# Получите количество эпох обучения
epochs = range(1, len(loss) + 1)

# Постройте график уменьшения ошибки для каждого целевого параметра
plt.plot(epochs, loss, 'b', label='Summory Loss')
plt.plot(epochs, loss_target2, 'r', label='Chord Loss')
plt.plot(epochs, loss_target3, 'g', label='Duration Loss')
plt.title('Уменьшение ошибки по каждому целевому параметру')
plt.xlabel('Эпохи')
plt.ylabel('Значение ошибки')
plt.legend()
plt.show()

## Сохранение модели

In [ ]:
modelChord2.save('chord2vecLSTM 2 branch.h5')